## Alpha Worm

Notes regarding loading the unity environment in python:
* **You can't use the Windows Subsystem for Linux**!
* The executable/environment has to build for the platform where you execute on


Zum Testen der Anbindung von Python zu Unity einfach `env = UnityEnvironment(file_name=None)`
 * Unity auf dem PC öffnen
 * Domain laden und auf Play drücken (Auf Konsolenoutput achten)
 * `Couldn't connect to trainer on port 5004 using API version 1.0.0. Will perform inference instead.`
    * Firewall off didn't change anything
    * run `mlagents-learn --help` in virtual environment `asp`


Steps done to run the environment from Jupyter
Hints:
* You have to install `tensorflow = 1.15.3` 
* Make sure that the environments are closed! If they are not closed probably the unity window would freeze
* You could also try to use another `worker_id = `
* The working directior doesn't have to be the root folder -> You can run an Unity Environment also in a subfolder e.g.
./
envs/
notebooks/run_env.ipynb

Run unity environment in Jupyter:

It is very import to close the environment. For the case that the window is grey/freezed try to execute `env.close()`. Probably you would see the environment working afterwards.

```python
env = UnityEnvironment('../src/envs/AlphaWormWin/alpha-worm-unity', no_graphics=False, worker_id=4)
env.close()
```


In [1]:
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from mlagents_envs.environment import UnityEnvironment
import mlagents

import logging 
logging.basicConfig(level=logging.DEBUG)

import sys
sys.path.insert(0, "../")

from utils.mlagent_utils import get_env

About the environment:
* action vector size of 9

In [2]:
win_env = "../envs/worm_dynamic_one_agent/win/UnityEnvironment"

env = get_env(win_env, True)

INFO:mlagents_envs.environment:Connected to Unity environment with package version 1.0.2-preview and communication version 1.0.0
INFO:mlagents_envs.environment:Connected new brain:
WormDynamic?team=0


In [11]:
# Random agent to test the environment connection

for idx in range(100):
    obs = env.reset()
    action = np.random.rand(1,9)
    obs, reward, done, info = env.step(action)

env.close()

INFO:mlagents_envs.environment:Environment shut down with return code 0 (CTRL_C_EVENT).


## DDPG Agent

In [10]:
# Dry DPPG run

%load_ext autoreload
%autoreload 2

from models.ddpg.ddpg import DDPGagent
from models.ddpg.OUNoise import OUNoise

win_env = "../envs/worm_dynamic_one_agent/win/UnityEnvironment"

env = get_env(win_env, True)

ddpg_agent = DDPGagent(env)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
INFO:mlagents_envs.environment:Connected to Unity environment with package version 1.0.2-preview and communication version 1.0.0
INFO:mlagents_envs.environment:Connected new brain:
WormDynamic?team=0


In [11]:
episodes = 1
training_steps = 500

batch_size = 128
rewards = []
avg_rewards = []

noise = OUNoise(env.action_space)

for episode in range(episodes):
    state = env.reset()
    noise.reset()
    episode_reward = 0
    
    for step in range(training_steps):
        action = ddpg_agent.get_action(state)
        action = noise.get_action(action, step)
        new_state, reward, done, _ = env.step(action) 
        ddpg_agent.memory_buffer.push(state, action, reward, new_state, done)
        
        
        if len(ddpg_agent.memory_buffer) > batch_size:
            ddpg_agent.update(batch_size)        
        
        state = new_state
        episode_reward += reward

env.close()

RuntimeError: size mismatch, m1: [128 x 57], m2: [66 x 256] at C:\w\b\windows\pytorch\aten\src\TH/generic/THTensorMath.cpp:41

In [9]:
env.close()

INFO:mlagents_envs.environment:Environment shut down with return code 0 (CTRL_C_EVENT).
